<a href="https://colab.research.google.com/github/leoxiang66/Knowledge-driven-spoken-dialogue/blob/main/result_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets

In [ ]:
!git lfs install
!git clone https://huggingface.co/datasets/Adapting/2022-GLOBAL-AI-CHALLENGE

In [ ]:
import json

with open('/content/2022-GLOBAL-AI-CHALLENGE/test.json', 'r') as f:
  data = json.load(f)

# Output: {'name': 'Bob', 'languages': ['English', 'French']}
print(data)

In [ ]:
import pandas as pd
klg_data = pd.read_csv('/content/2022-GLOBAL-AI-CHALLENGE/knowledge_data.csv')
klg_data

In [ ]:
klg_data[klg_data.entity=='调王生']['knowledge'].values[0]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

tokenizer = AutoTokenizer.from_pretrained("Adapting/Knowledge-Driven-Dialogue")
model_5 = AutoModelForSeq2SeqLM.from_pretrained("Adapting/Knowledge-Driven-Dialogue", revision = '5e2267251ec1555e81f9ed6f090e1f70355ff1c8')

pipe_5 = pipeline('text2text-generation',tokenizer=tokenizer,model=model_5,device=0)
pipe_truncate = pipeline('text2text-generation',tokenizer=tokenizer,model=model_5,device=0, truncation=True, max_length=1024)

In [ ]:
ALL_ENEITIES = list(klg_data['entity'].values)
ALL_ENEITIES

In [ ]:
for i in ALL_ENEITIES:
    if not isinstance(i,str):
        print(i)
        print(type(i))

In [ ]:
def entity_detection(usr):
    detected_ent = []
    for ent in ALL_ENEITIES:
        if len(str(ent)) <= 1:
            continue
        if str(ent) in usr:
            detected_ent.append(ent)
    return detected_ent

In [ ]:
entity_detection('你知道上元日吗')

# Iteration

In [ ]:
result = dict()
for i in range(400):
    # 第i个样本
    sample = data[f'{i+1}'] # list of dicts
    chat_history = ''
    for j in range(0,len(sample)-1,2):
        last_usr_uttr = sample[j]['message']
        last_sys_uttr = sample[j+1]['message']
        chat_history += f'[usr] {last_usr_uttr} [sys] {last_sys_uttr} '

    last_usr_uttr = sample[-1]['message']
    chat_history += f'[usr] {last_usr_uttr}'

    entity = entity_detection(sample[0]['message'])
    if len(entity)> 1:
        # print(i)
        # print(entity)
        knowledge = klg_data[klg_data.entity==max(entity,key=len)]['knowledge'].values[0]
    if len(entity)> 0:
        knowledge = klg_data[klg_data.entity==entity[0]]['knowledge'].values[0]
    else:
        knowledge = ''
    
    input = f'[klg] {knowledge} {chat_history}'
    if len(input)<1440:
        response = pipe_5(input)[0]['generated_text'].replace(' ','')
    else:
        response = pipe_truncate(input)[0]['generated_text'].replace(' ','')
    
    print(f'''{i+1}:
    input: {input}
    response: {response}
    ------------------------------------------------------------------------------------------
    ''')

    if len(entity) == 0:
        result[f'{i+1}'] = {
            'message': response
        }

    else:
        result[f'{i+1}'] = {
             'message': response,
             "attrs":[
                 {"attrname":"None",
                "attrvalue":"None",
                "name":max(entity,key=len)}
             ]
        }


In [ ]:
import json
with open('result.json', 'w', encoding='utf8') as fp:
    json.dump(result, fp,ensure_ascii=False)